In [1]:
# @title PIP
%pip install opencv-python==4.8.0.76
%pip install pytesseract==0.3.9

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.

                                              0.0/38.1 MB ? eta -:--:--
                                              0.3/38.1 MB 6.3 MB/s eta 0:00:06
     -                                        1.2/38.1 MB 15.0 MB/s eta 0:00:03
     --                                       2.2/38.1 MB 17.3 MB/s eta 0:00:03
     ---                                      3.2/38.1 MB 18.6 MB/s eta 0:00:02
     ----                                     4.5/38.1 MB 20.5 MB/s eta 0:00:02
     -----                                    5.7/38.1 MB 21.5 MB/s eta 0:00:02
     -------                                  7.3/38.1 MB 23.4 MB/s eta 0:00:02
     ---------                                8.9/38.1 MB 24.8 MB/s eta 0:00:02
     -----------                             10.8/38.1 MB 29.7 MB/s eta 0:00:01
     ------------                            12.6/38.1 MB 32.7 MB


[notice] A new release of pip is available: 23.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable
                                              0.0/2.6 MB ? eta -:--:--
     ---                                      0.2/2.6 MB 6.9 MB/s eta 0:00:01
     -------------------                      1.3/2.6 MB 16.5 MB/s eta 0:00:01
     ---------------------------------------  2.6/2.6 MB 23.8 MB/s eta 0:00:01
     ---------------------------------------- 2.6/2.6 MB 18.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
from google.colab import drive6
drive.mount('/content/drive')

In [ ]:
# @title TESTES ):D
main()





##Versão nova

In [18]:
import pytesseract
import regex as re
import numpy as np
import cv2
#from google.colab.patches import cv2_imshow
from PIL import ImageFont, ImageDraw, Image
import re
import os

pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'


def list_files_in_directory(folder_path):
  old_parts = ["primeiroValor","primeiroValor","primeiroValor","primeiroValor"]
  files_array = []
  document_array = []
  for root, dirs, files in os.walk(folder_path):
    for file in files:
        
      file_path = os.path.join(root, file)
      
      parts = file.split('-')
      print(parts)
    
      if parts[0] == old_parts[0] and parts[1] == old_parts[1] and parts[2] == old_parts[2] and parts[3] == old_parts[3]:
        document_array.append(file_path)
      else:
        if old_parts[0] != "primeiroValor":
          files_array.append(document_array)
          document_array = []
        document_array.append(file_path)

      old_parts = parts

  # print(files_array)
  return files_array

def descobre_prestadora(documento):
  if re.findall('^[Cc][Ll][Aa][Rr][Oo]', documento,  re.MULTILINE):
        prestadora = 'Claro'
  elif re.findall('^[Tt][Ii][Mm]', documento,  re.MULTILINE):
        prestadora = 'Tim'
  elif re.findall('^[Vv][Ii][Vv][Oo]', documento, re.MULTILINE):
        prestadora = 'Vivo'
  else:
        prestadora = None

  return prestadora

def extrair_info(documento, prestadora):
  tipo_conta = None
  n_conta = None
  mes_referencia = None
  vencimento = None
  emisao = None
  cpf = None
  periodo = None
  valor_vencimento = None
  
  if prestadora == 'Vivo':
    #PIPELINE VIVO
    tipo_conta = 'Telefonia'

    n_conta = re.findall(r'[Cc][Oo][Nn][Tt][Aa](.*[\r\n].*)(\b\d{9}\b)', documento, re.MULTILINE)

    #TEM VERSÃO COM TEXTO NO DIA DO MES , VERIFICAR ISSO
    #Notas para o paper, esse regex foi complexa de confecionar, a diferença entre o bloco de texto que a geente procurava era "referencia:" ou "referencia "
    mes_referencia = re.findall(r"[Rr][Ee][Ff][Ee][Rr][Êê|Ee][Nn][Cc][Ii][Aa][\:\s|\s](.*?)(0[1-9]|1[0-2])/(19|20)\d{2}", documento, re.MULTILINE)
    #mes_referencia = pre_mes_referencia[1]

    #VENCIMENTO PODE NÃO SER POSSIVEL
    vencimento =  re.findall(r'((.|\n)*?)\b(0[1-9]|1[0-2])\/(19|20[0-9]{2})\b', documento, re.MULTILINE)

    emisao = re.findall(r'[Dd][ÁáAa][Tt][Aa]\s[Dd][Ee] [Ee][Mm][IiÍí][Ss][Ss][ÃãAaÃa][OoOo]\s(0[1-9]|[12][0-9]|3[01])/(0[1-9]|1[0,1,2])/(19|20)\d{2}', documento, re.MULTILINE)

    cpf = encontra_cpf_cnpj(documento)

    periodo = re.findall(r'(P|p)(E|e)(R|r)(Í|í)(O|o)(D|d)(O|o)\s\:.*?\n((0[1-9]|[12][0-9]|3[01])\/(0[1-9]|1[0,1,2])\/(19|20)\d{2})', documento, re.MULTILINE)

    valor_vencimento = None


  elif prestadora == 'Claro':

    regex_n_conta = r'[Cc][Oo][Nn][Tt][Aa](.*[\r\n].*)(\b\d{9}\b)'

    # Regex para encontrar a data de vencimento
    regex_data_vencimento = r'\b(\d{2}/\d{2}/\d{4})\b'

    # Regex para encontrar a data de início
    regex_data_inicio = r'\b(\d{2}/\d{2}/\d{4})\b'

    # Regex para encontrar o valor total
    regex_valor_total = r'\b(\d+,\d{2})\b'


    #PIPELINE CLARO

    tipo_conta = 'Telefonia'



    pre_n_conta = re.findall(regex_n_conta, documento, re.MULTILINE)
    n_conta = pre_n_conta#[0][1]

    pre_mes_referencia = re.findall(regex_data_vencimento, documento, re.MULTILINE)
    mes_referencia = pre_mes_referencia#[0]

    vencimento =  re.findall(r'[Vv][Ee][Nn][Cc][Ii][Mm][Ee][Nn][Tt][Oo]\s(0[1-9]|[12][0-9]|3[01])/(0[1-9]|1[0,1,2])/(19|20)\d{2}', documento, re.MULTILINE)

    emisao = re.findall(regex_data_inicio, documento, re.MULTILINE)

    cpf = encontra_cpf_cnpj(documento)

    periodo = None

    valor_vencimento = re.findall(regex_valor_total, documento, re.MULTILINE)

  elif prestadora == 'Tim':
    #PIPELINE TIM
    pass

  print(prestadora)
  print(tipo_conta)
  print(n_conta)
  print(mes_referencia)
  print(vencimento)
  print(emisao)
  print(cpf)
  print(periodo)
  print(valor_vencimento)

  return prestadora, tipo_conta, n_conta, mes_referencia, vencimento, emisao, cpf, periodo, valor_vencimento

def encontra_cpf_cnpj(documento):
  tipo = 'CNPJ'
  dado = re.findall(r"[Cc][Pp][Ff]\/[Cc][Nn][Pp][Jj].*?:.*?(?=\d)(.*?(?=\d)([0-9]{3}\.?[0-9]{3}\.?[0-9]{3}\-?[0-9]{2}))", documento, re.MULTILINE)
  return dado #dado[0][0]

def main(arg):
  fonte = '/content/drive/MyDrive/Colab Notebooks/Fonte/Calibri/Calibri.ttf'


  #paths = ['C:\_CONTAS_COLETADAS\separados\claro-03-2023-johnathan\claro-03-2023-johnathan-page1.png']
  #paths = ['/content/page0.png','/content/page1.png','/content/page2.png']
  #paths = ['/content/drive/MyDrive/_TCC/Contas_coletadas/output-final/vivo-08-2023-cintia-page1.png']
  path = arg
  paths = list_files_in_directory(path)
  documento = ''
  array_documento = []
  for x in paths:
    documento = ''
    for i, pdfs in enumerate(x):
      img = cv2.imread(x[i])
      convertido = pytesseract.image_to_string(img, lang = 'por', config='--psm 3')
      documento += convertido
      # print(documento)
    array_documento.append(documento)
  #print(array_documento)
  #PEGA PRESTADORA
  
  
  infos = []
  
  for documento in array_documento:
    prestadora = descobre_prestadora(documento)
    info = extrair_info(documento, prestadora)
    infos.append(info)
  print(infos)
  return infos
  # Método de instância para exibir informações da conta


In [19]:

all_data = []

data = main('C:\_CONTAS_COLETADAS')


all_data.append(data)

['claro', '03', '2023', 'ana', 'page1.png']
['claro', '03', '2023', 'ana', 'page2.png']
['claro', '03', '2023', 'johnathan', 'page1.png']
['claro', '04', '2023', 'ana', 'page1.png']
['claro', '04', '2023', 'ana', 'page2.png']
['claro', '04', '2023', 'ana', 'page3.png']
['claro', '04', '2023', 'ana', 'page4.png']
['claro', '05', '2023', 'ana', 'page1.png']
['claro', '05', '2023', 'ana', 'page2.png']
['claro', '05', '2023', 'johnathan', 'page1.png']
['claro', '06', '2023', 'ana', 'page1.png']
['claro', '06', '2023', 'ana', 'page2.png']
['claro', '06', '2023', 'fernanda', 'page1.png']
['claro', '06', '2023', 'fernanda', 'page2.png']
['claro', '06', '2023', 'johnathan', 'page1.png']
['claro', '06', '2023', 'johnathan', 'page2.png']
['claro', '06', '2023', 'johnathan', 'page3.png']
['claro', '06', '2023', 'johnathan', 'page4.png']
['claro', '07', '2023', 'ana', 'page1.png']
['claro', '07', '2023', 'ana', 'page2.png']
['claro', '07', '2023', 'gabriela', 'page1.png']
['claro', '07', '2023', '

In [ ]:

import pandas as pd


df = pd.DataFrame(all_data, columns=["prestadora", "tipo_conta", "n_conta", "mes_referencia", "vencimento", "emisao", "cpf", "periodo", "valor_vencimento"])


excel_file = "dados_contas.xlsx"


df.to_excel(excel_file, index=False, engine='openpyxl')

print(f'Dados foram salvos em {excel_file}')